# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [2]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Study of the effect of environment on AGN activity in HR5 simulation	Talk		University of Hull, UK	2023-05-15		
Studying Galaxy Evolution using Hydro-Dynamical Simulations	Talk		Nehru Planetarium, New Delhi	2023-02-16		
Impact Of AGN emission on estimated star formation rate using IR luminosity of galaxies in HR5	Talk		Mapping the invisible Universe, at Lorentz Center, Leiden, The Netherlands	2022-08-29 to 2022-09-02		
Effects of local environment on AGN activity in HR5	Talk		The 10th KIAS Workshop on Cosmology and Structure Formation at KIAS	2022-10-24 to 2022-10-28		
Study of filament galaxies in EAGLE simulation	Talk		LSST@ASIA: Exploring the Wide, Fast, Deep Universe conference	at UNSW, Sydney, Australia	2019-05-20 to 2019-05-23		
Study of filament galaxies in EAGLE simulation	Talk		XXXVII Meeting of Astronomical Society of India, Christ (Deemed to be University), Bengaluru, India	2019-02-18 to 2019-02-22		
Caught in the w

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [10]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Study of the effect of environment on AGN acti...,Talk,NaN,University of Hull UK,2023-05-15,NaN,NaN,NaN
1,Studying Galaxy Evolution using Hydro-Dynamica...,Talk,NaN,Nehru Planetarium New Delhi,2023-02-16,NaN,NaN,NaN
2,Impact Of AGN emission on estimated star forma...,Talk,NaN,Mapping the invisible Universe at Lorentz Cent...,2022-08-29 to 2022-09-02,NaN,NaN,NaN
3,Effects of local environment on AGN activity i...,Talk,NaN,The 10th KIAS Workshop on Cosmology and Struct...,2022-10-24 to 2022-10-28,NaN,NaN,NaN
4,Study of filament galaxies in EAGLE simulation,Talk,NaN,LSST@ASIA: Exploring the Wide Fast Deep Univer...,at UNSW Sydney Australia 2019-05-20 to 2019...,NaN,NaN,NaN
5,Study of filament galaxies in EAGLE simulation,Talk,NaN,XXXVII Meeting of Astronomical Society of Indi...,NaN,NaN,NaN,NaN
6,Caught in the web: a tale of filament galaxies,Poster,NaN,IAU Symposium 341: PanModel2018: Challenges in...,2018-11-12 to 2018-11-16,NaN,NaN,NaN
7,Ultraviolet and optical view of galaxies in Co...,Talk,NaN,Young Astronomer's Meet (YAM) Physical Researc...,NaN,NaN,NaN,NaN
8,An analytical study of ram pressure stripping ...,Talk,NaN,Franco-Indian Astronomy School: From Re-ioniza...,2018-02-11 to 2018-02-17,NaN,NaN,NaN
9,Ram Pressure Stripping: An analytical approach,Poster,NaN,XXXVI Meeting of Astronomical Society of India...,2018-02-05 to 2018-02-09,NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2012-03-01-talk-1.md	  2014-02-01-talk-2.md
2013-03-01-tutorial-1.md  2014-03-01-talk-3.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
